In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('OH.csv') # Large file

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6165997 entries, 0 to 6165996
Data columns (total 26 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   state                  object 
 2   stop_date              object 
 3   stop_time              object 
 4   location_raw           float64
 5   county_name            object 
 6   county_fips            float64
 7   fine_grained_location  object 
 8   police_department      float64
 9   driver_gender          object 
 10  driver_age_raw         float64
 11  driver_age             float64
 12  driver_race_raw        object 
 13  driver_race            object 
 14  violation_raw          object 
 15  violation              object 
 16  search_conducted       bool   
 17  search_type_raw        object 
 18  search_type            object 
 19  contraband_found       object 
 20  stop_outcome           object 
 21  is_arrested            bool   
 22  lat               

#### Remove columns that are uninformative or all nulls

In [10]:
relevant_cols = ['stop_date','stop_time','location_raw','driver_gender','driver_race','driver_race_raw','violation',
                 'search_conducted','contraband_found','is_arrested','drugs_related_stop']

data = df[relevant_cols].copy()

#### Replace time with hour and date with a month and year column

In [11]:
data['stop_time'] = data['stop_time'].apply(lambda x: x.split(':')[0])
data['stop_year'] = data['stop_date'].apply(lambda x: x.split('-')[0])
data['stop_month'] = data['stop_date'].apply(lambda x: x.split('-')[1])
data.drop(['stop_date', 'stop_time'], inplace=True, axis=1)

#### Keep 20 top locations and populate rest with None to be imputed later

In [12]:
top_locations = list(data['location_raw'].value_counts().keys()[:20])
data['location_raw'] = data['location_raw'].apply(lambda x: int(x) if x in top_locations else None)

#### Joining the two driver race columns and uniting the information contained in both

In [13]:
data['driver_race'].loc[data['driver_race']=='Other'] = data['driver_race_raw'].loc[data['driver_race']=='Other']
data.drop('driver_race_raw',axis=1,inplace=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [22]:
def violation_mapping(x):
    '''
    Maps violations from word descriptors to categories with decreasing severity (0 is most severe)
    '''
    severity_map = {
        'dui':0,
        'speeding':1,
        'stop sign/light':2,
        'license':3,
        'cell phone':4,
        'paperwork':5,
        'registration/plates':6,
        'safe movement':7,
        'seat belt':8,    
        'equipment':9,
        'lights':10,
        'truck':11,
        'other':12,
        'other (non-mapped)':13
    }
    
    viols = []
    for k,v in severity_map.items():
        if (k in x.lower()):
            viols.append(v)  
    return min(viols)

In [26]:
data['violations_numbered'] = data['violation'].fillna('other (non-mapped)').apply(violation_mapping)
data.drop('violation',axis=1,inplace=True)

In [28]:
categorical = list(data.columns)
categorical.remove('is_arrested')

In [29]:
def replace_nan_categorical(df,column):
    prob = dict(df[column].value_counts()/len(df))
    keys = list(prob.keys())
    sum_prob = sum(prob.values())
    for k,v in prob.items():
        prob[k] = prob[k]/sum_prob
    prob_list = list(prob.values())
    to_fill = np.random.choice(keys, len(df[column].loc[df[column].isnull()]), p = prob_list) 
    df[column].loc[df[column].isnull()] = to_fill   
    return df

In [30]:
for c in categorical:
    data = replace_nan_categorical(data,c)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [41]:
data['is_arrested'] = data['is_arrested'].apply(lambda x: 1 if x == True else 0)

In [42]:
dummies = pd.get_dummies(data,columns=categorical)
dummies.to_csv('OH_cleaned.csv',index=False)
sampled = dummies.sample(frac=0.1,random_state=2018).reset_index(drop=True)
sampled.to_csv('OH_sampled_cleaned.csv',index=False)